In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime as dt
from sklearn.preprocessing import MinMaxScaler


confirmed_cases = pd.read_csv('Dados/time_series_2019-ncov-Confirmed.csv')
confirmed_deaths = pd.read_csv('Dados/time_series_2019-ncov-Deaths.csv')
confirmed_recovered = pd.read_csv('Dados/time_series_2019-ncov-Recovered.csv')

In [2]:
confirmed_recovered

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,2,2,...,35,35,35,35,41,42,42,42,42,44
1,NaN,Japan,36.0000,138.0000,0,0,0,0,1,1,...,118,118,118,144,144,144,150,191,232,235
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,97,105,105,109,114,114,114,124,140,144
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,26,35,42,42,49,60,75,87,114,139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,NaN,Dominica,15.4150,-61.3710,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
483,NaN,Grenada,12.1165,-61.6790,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
484,NaN,Mozambique,-18.6657,35.5296,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
485,NaN,Syria,34.8021,38.9968,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
confirmed_cases['Province/State']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
482    NaN
483    NaN
484    NaN
485    NaN
486    NaN
Name: Province/State, Length: 487, dtype: object

In [4]:
class MLP():
    def __init__(self):
        pass
    def NormalizeData(self,df):
        self.scaler = MinMaxScaler(feature_range=(-1, 1))
        self.normalized = self.scaler.fit_transform(df)
        print(self.normalized)
        
    def Denormalize(self,dfNormalized):
        pass
    
    
    def PrepareData(self,timesteps):
        i = 0
        self.X = []
        self.Y = []
        while i in range(len(self.normalized - timesteps)):
            input_index = i + timesteps
            label_index = input_index + 1
            if(label_index < len(self.normalized)):
                self.X.append(self.normalized[i:input_index,:])
                self.Y.append(self.normalized[input_index:label_index,2])
            i+=1
        self.X = np.array(self.X)
        self.Y = np.array(self.Y)
        print(self.X)
        print(self.Y)
    
    '''
    def Prepare_Data(self,dataset):
        x = dataset.drop(columns=['Total_Deaths'])
        self.X = x.to_numpy()
        y = dataset['Total_Deaths']
        self.Y = y.to_numpy()
        self.Y = self.Y.astype(float)
    '''
    def Build(self,janela,nmr_parametros):
        self.model = keras.Sequential()
        self.model.add(keras.layers.LSTM(216, input_shape=(janela, nmr_parametros), return_sequences=True))
        self.model.add(keras.layers.LSTM(128, return_sequences=True))
        self.model.add(keras.layers.LSTM(64, return_sequences=False))
        self.model.add(keras.layers.Dense(16, activation="relu", kernel_initializer="uniform"))
        self.model.add(keras.layers.Dense(1, activation="linear"))
                       
    def RMSE(self,y_true,y_pred):
        print(y_true)
        return tf.cast(keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true))),tf.float32)

    def Fit(self):
        self.model.compile(loss=self.RMSE,optimizer=keras.optimizers.Adam(),metrics=['mae',self.RMSE])
        self.history = self.model.fit(x=self.X,y=self.Y,epochs=50,shuffle=False,verbose=True)
    def Predict(self,data):
        result = self.model.predict(data,verbose=True)
        return result
        

In [5]:
class Data():
    def __init__(self):
        pass
    def PreparaData(self,confirmed,deaths,recovered):
        confirmed = confirmed.drop(columns=['Province/State','Country/Region','Lat','Long'])
        deaths = deaths.drop(columns=['Province/State','Country/Region','Lat','Long'])
        recovered = recovered.drop(columns=['Province/State','Country/Region','Lat','Long'])
        timesteps = []
        total_infetados = []
        total_Days = []
        total_Deaths = []
        total_Recovered = []
        self.new_dataset = pd.DataFrame()
        Ground_Zero = dt.strptime('12/31/19','%m/%d/%y')
        for cols in confirmed.columns:
            timesteps.append(cols)
            total_infetados.append(confirmed[cols].sum())
            current_date = dt.strptime(cols,'%m/%d/%y')
            days_Gone = current_date - Ground_Zero
            total_Days.append(int(days_Gone.days))
        for cols in deaths.columns:
            total_Deaths.append(deaths[cols].sum())
        for cols in recovered.columns:
            total_Recovered.append(recovered[cols].sum())
        self.new_dataset['Total_Cases'] = total_infetados
        self.new_dataset['Total_Recovered'] = total_Recovered
        self.new_dataset['Total_Deaths'] = total_Deaths
        self.new_dataset['Days_Gone'] = total_Days
        pd.DataFrame.to_csv(self.new_dataset,'Dados/new_dataset.csv')

In [6]:
data = Data()
data.PreparaData(confirmed_cases,confirmed_deaths,confirmed_recovered)
print(data.new_dataset)

    Total_Cases  Total_Recovered  Total_Deaths  Days_Gone
0           555               28            17         22
1           653               30            18         23
2           941               36            26         24
3          1434               39            42         25
4          2118               52            56         26
..          ...              ...           ...        ...
56       214910            83207          8733         78
57       242708            84854          9867         79
58       272166            87256         11299         80
59       304524            91499         12973         81
60       335955            97704         14632         82

[61 rows x 4 columns]


In [7]:
model = MLP()
model.Build(7,4)
model.NormalizeData(data.new_dataset)
model.PrepareData(7)

[[-1.         -1.         -1.         -1.        ]
 [-0.99941562 -0.99995905 -0.99986315 -0.96666667]
 [-0.99769827 -0.99983619 -0.99876839 -0.93333333]
 [-0.9947585  -0.99977477 -0.99657886 -0.9       ]
 [-0.99067979 -0.99950858 -0.99466302 -0.86666667]
 [-0.98585569 -0.9993243  -0.99110503 -0.83333333]
 [-0.9700477  -0.99838241 -0.98439959 -0.8       ]
 [-0.96654144 -0.99799337 -0.9841259  -0.76666667]
 [-0.9542099  -0.99764528 -0.97892576 -0.73333333]
 [-0.94411449 -0.99602768 -0.97317824 -0.7       ]
 [-0.93152654 -0.99475818 -0.96688334 -0.66666667]
 [-0.90320811 -0.99090872 -0.95278823 -0.63333333]
 [-0.8847585  -0.98781686 -0.94403011 -0.6       ]
 [-0.86084079 -0.98312789 -0.93499829 -0.56666667]
 [-0.83852117 -0.97755846 -0.9251454  -0.53333333]
 [-0.81954681 -0.97012572 -0.9155662  -0.5       ]
 [-0.79823494 -0.95939637 -0.90393431 -0.46666667]
 [-0.78196184 -0.94700848 -0.89202874 -0.43333333]
 [-0.76389386 -0.93414964 -0.87834417 -0.4       ]
 [-0.74831843 -0.91977558 -0.86

In [8]:
model.Fit()

Tensor("dense_1_target:0", shape=(None, None), dtype=float32)
Tensor("dense_1_target:0", shape=(None, None), dtype=float32)
Train on 53 samples
Epoch 1/50
Tensor("Cast_1:0", shape=(None, 1), dtype=float32)
Tensor("metrics/RMSE/Cast:0", shape=(None, 1), dtype=float32)
Tensor("Cast_1:0", shape=(None, 1), dtype=float32)
Tensor("metrics/RMSE/Cast:0", shape=(None, 1), dtype=float32)
53/53 [==============================] - 5s 99ms/sample - loss: 0.6689 - mae: 0.6456 - RMSE: 0.6302
Epoch 2/50
53/53 [==============================] - 0s 981us/sample - loss: 0.6400 - mae: 0.6173 - RMSE: 0.6036
Epoch 3/50
53/53 [==============================] - 0s 849us/sample - loss: 0.5980 - mae: 0.5760 - RMSE: 0.5647
Epoch 4/50
53/53 [==============================] - 0s 943us/sample - loss: 0.5251 - mae: 0.5039 - RMSE: 0.4972
Epoch 5/50
53/53 [==============================] - 0s 774us/sample - loss: 0.3961 - mae: 0.3718 - RMSE: 0.3790
Epoch 6/50
53/53 [==============================] - 0s 736us/sample - l

In [15]:
prediction = model.Predict(model.X[:-1])
#value = model.scaler.inverse_transform(prediction)[:, [2]]
print(prediction)

52/52 [==============================] - 0s 269us/sample
[[-0.9901514 ]
 [-0.98262274]
 [-0.97462523]
 [-0.9661471 ]
 [-0.9571931 ]
 [-0.94760656]
 [-0.9373422 ]
 [-0.9270333 ]
 [-0.9155065 ]
 [-0.90335137]
 [-0.89027923]
 [-0.87639034]
 [-0.86304617]
 [-0.84946704]
 [-0.8360602 ]
 [-0.8229558 ]
 [-0.80949557]
 [-0.7956063 ]
 [-0.7807367 ]
 [-0.76500195]
 [-0.7481522 ]
 [-0.72970784]
 [-0.7079234 ]
 [-0.6926814 ]
 [-0.6795908 ]
 [-0.6669595 ]
 [-0.654605  ]
 [-0.6423301 ]
 [-0.63064253]
 [-0.61860144]
 [-0.6062119 ]
 [-0.59292084]
 [-0.57991743]
 [-0.5660845 ]
 [-0.55256945]
 [-0.53789306]
 [-0.5217195 ]
 [-0.50343084]
 [-0.48238844]
 [-0.45879486]
 [-0.43217042]
 [-0.401861  ]
 [-0.36719367]
 [-0.32650852]
 [-0.2788703 ]
 [-0.22369874]
 [-0.15791927]
 [-0.08221368]
 [ 0.09037559]
 [ 0.23571739]
 [ 0.39082935]
 [ 0.5716967 ]]
